In [1]:
import rdata
import pandas as pd
import glob
import re
import os 
import dask.dataframe as dd
import statistics
import numpy as np

from statistics import mean

from sklearn.metrics import r2_score

In [3]:
# Read the matrix of measured values used to create the RefFIT model
RefFIT_Measurement_Path = r"data\RefFIT_MeasurementValue.csv" 
RefFIT_Measurement_Matrix = dd.read_csv(RefFIT_Measurement_Path,assume_missing=True, sample=1000000).compute()

RefFIT_Pridiction_Path = r"data\RefFIT_PridictionValue.csv"
RefFIT_Pridiction_Matrix = dd.read_csv(RefFIT_Pridiction_Path , assume_missing=True, sample=1000000).compute()

In [4]:
# Erase trailing content from column name (Gene) of predicted value
Analysis_Gene_List = list(RefFIT_Pridiction_Matrix.columns)
Analysis_Gene_List = [item[:-len('.radiation')] if item.endswith('.radiation') else item[:-len('.temperature')] if item.endswith('.temperature') else item for item in Analysis_Gene_List]
RefFIT_Pridiction_Matrix.columns = Analysis_Gene_List

# Match the number of genes in the matrix of measured values and the matrix of predicted values.
RefFIT_Measurement_Matrix = RefFIT_Measurement_Matrix[Analysis_Gene_List]

In [5]:
df_SD_List = []

for Gene in Analysis_Gene_List:
    
    Measurement_Expression_List = list(RefFIT_Measurement_Matrix[Gene])
    Pridiction_Expression_List = list(RefFIT_Pridiction_Matrix[Gene])
    
    residuals = np.array(Measurement_Expression_List) - np.array(Pridiction_Expression_List)

    std_deviation = np.std(residuals)

    df_SD_List.append([Gene , std_deviation])
    

df_SD = pd.DataFrame(df_SD_List , columns=["Gene" , "SD"])

In [6]:
df_SD.to_csv(r"SD.csv" , index = False)